# Sigma_density: M Dwarfs with Spectroscopic Constraints

Calculating density constraints for M Dwarf KOIs with spectroscopic constraints on properties.

1. From spectroscopy, we know Temperature, Mass, and Radius.
2. Which isochrones (stellar models, with Temperature, Mass, and Radius as parameters) fit above star?
3. Now take into account GAIA distance for the star. From this, we know the Luminosity.
4. Look back at stellar models - which isochrone fits the Temperature, Mass Radius, and Luminority (from GAIA)?

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from astropy.table import Table

plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [62]:
headers = ['KOI', 'KIC', 'Teff', 'ETeff', 'eTeff', 'Fe/H', 'e_Fe/H', 'M/H', 'e_M/H', 'SpType', 'Mstar', 'e_Mstar', 'Rstar', 'e_Rstar', 'Dis', 'Fe/H-T', 'e_Fe/H-T', 'Fe/H-M', 'e_Fe/H-M', 'M/H-M', 'e_M/H-M']

For now, removed any rows with missing data and saved them into "muirhead_data_no_missing_data.txt".

In [63]:
muirhead_data = pd.read_csv("/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/muirhead_data_no_missing_data.txt", sep=" ")

In [64]:
isochrones = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/isochrones_sdss_spitzer_lowmass.dat', sep='\s\s+', engine='python')

#### First, let's work with muirhead_data[0]: KIC 5868793.

1. Take Teff, Mass, and Radius.
2. Which isochrones fit these Teff, Mass, and Radius?

#### Reading Gaia-Kepler data (luminosities)

In [237]:
dat = Table.read('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Kepler-Gaia/kepler_dr2_4arcsec.fits', format='fits')
df = dat.to_pandas()

/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_result_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_modality_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [238]:
muirhead_gaia = df[df['kepid'].isin(list(muirhead_data.KIC))]
muirhead_gaia = muirhead_gaia.reset_index()
muirhead_gaia.rename(columns={"index": "KIC"})

#muirhead_comb: planet hosts with spectroscopic data + Gaia/Kepler data in one table
muirhead_comb = pd.concat([muirhead_data, muirhead_gaia], axis=1)

In [239]:
test_planet = muirhead_comb.loc[muirhead_comb['KIC'] == 8733898]

In [240]:
test_planet.lum_val

1   NaN
Name: lum_val, dtype: float32

In [241]:
#muirhead_comb["KIC"]

In [242]:
#muirhead_comb["lum_val"]

In [243]:
#pd.options.display.max_colwidth = 100

In [244]:
#test_planet['datalink_url']

In [245]:
def fit_isochrone(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia data for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ONLY ON SPECTROSCOPY.
       """            
    
    iso_fits_final = list()
    
    #test each star in spectroscopy sample:
    #for i in tqdm(range(len(muirhead_comb))):
    for i in tqdm(range(1)):

        iso_fits = pd.DataFrame() 

        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]

        #test each stellar model to see if it falls within error bars:
        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])

        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]

        iso_fits_final.append(iso_fits)

    return iso_fits_final


In [246]:
#isos = fit_isochrone(muirhead_comb, isochrones)

In [247]:
#isochrones['logg']

In [248]:
#for i in range(len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str([i]) + ".csv")

In [249]:
#for i in range(12, len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str(isos[i]['KIC'].iloc[0]) + ".csv")

In [251]:
def fit_isochrone_lum(data, isochrones, gaia_lum=True):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ON SPECTROSCOPY AND GAIA LUMINOSITY.
       """    
    
    iso_fits_final = list()
    
    #for i in tqdm(range(len(muirhead_comb))):
    for i in range(1):
        
        iso_fits = pd.DataFrame() 
    
#         Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
#         Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
#         Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]
#         lum_range = [data.lum_val[i]-data.lum_percentile_lower[i], data.lum_val[i]+data.lum_percentile_lower[i]]

        Teff_range = [float(data.Teff)-float(data.eTeff), float(data.Teff)+float(data.ETeff)]
        Mstar_range = [float(data.Mstar)-float(data.e_Mstar), float(data.Mstar)+float(data.e_Mstar)]
        Rstar_range = [float(data.Rstar)-float(data.e_Rstar), float(data.Rstar)+float(data.e_Rstar)]
        lum_range = [float(data.lum_val)-float(data.lum_percentile_lower), float(data.lum_val)+float(data.lum_percentile_lower)]

        print(Teff_range)
        print(Mstar_range)
        print(Rstar_range)
        print(lum_range)
        
        for j in tqdm(range(len(isochrones))):
            if gaia_lum==True:
                if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1] and lum_range[0] < 10**isochrones.logl_ls[j] < lum_range[1]:
                    iso_fits = iso_fits.append(isochrones.loc[[j]])
            
            if gaia_lum==False:
                if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                    iso_fits = iso_fits.append(isochrones.loc[[j]])
        
        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]
        
        iso_fits_final.append(iso_fits)
            
    return iso_fits_final

In [252]:
iso_lums = fit_isochrone_lum(test_planet, isochrones, gaia_lum=False)


  0%|          | 1543/738479 [00:00<00:47, 15398.30it/s]

[3137.0, 3244.0]
[0.14, 0.18]
[0.16, 0.19999999999999998]
[nan, nan]


100%|██████████| 738479/738479 [00:51<00:00, 14283.68it/s]


In [253]:
for i in range(len(iso_lums)):
   try:
       iso_lums[i].to_csv("jan29_21_iso_lums_" + '8733898' + ".csv")
   except IndexError:
       pass

In [254]:
isodf = pd.read_csv(r'/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/photoeccentric/notebooks/jan29_21_iso_lums_8733898.csv')

In [255]:
mstar = isodf["mstar"].mean()
mstar_err = isodf["mstar"].std()

rstar = isodf["radius"].mean()
rstar_err = isodf["radius"].std()

### Calculate rho

In [259]:
#Let's just do 1 star for now
ntargs = 1

In [260]:
# def find_density_dist_symmetric(ntargs, masses, masserr, radii, raderr):
#     """Gets symmetric stellar density distribution for stars.
#     Symmetric stellar density distribution = Gaussian with same sigma on each end.

#     Parameters
#     ----------
#     ntargs: int
#         Number of stars to get distribution for
#     masses: np.ndarray
#         Array of stellar masses (solar mass)
#     masserr: np.ndarray
#         Array of sigma_mass (solar mass)
#     radii: np.ndarray
#         Array of stellar radii (solar radii)
#     raderr: np.ndarray
#         Array of sigma_radius (solar radii)

#     Returns
#     -------
#     rho_dist: np.ndarray
#         Array of density distributions for each star in kg/m^3
#         Each element length 1000
#     mass_dist: np.ndarray
#         Array of symmetric Gaussian mass distributions for each star in kg
#         Each element length 1000
#     rad_dist: np.ndarray
#         Array of symmetric Gaussian radius distributions for each star in m
#         Each element length 1000
#     """

#     smass_kg = 1.9885e30  # Solar mass (kg)
#     srad_m = 696.34e6     # Solar radius (m)

#     rho_dist = np.zeros((ntargs, 1000))
#     mass_dist = np.zeros((ntargs, 1000))
#     rad_dist = np.zeros((ntargs, 1000))

#     for star in tqdm(range(ntargs)):

#         rho_temp = np.zeros(1000)
#         mass_temp = np.zeros(1000)
#         rad_temp = np.zeros(1000)

#         mass_temp = np.random.normal(masses[star]*smass_kg, masserr[star]*smass_kg, 1000)
#         rad_temp = np.random.normal(radii[star]*srad_m, raderr[star]*srad_m, 1000)

#         #Add each density point to rho_temp (for each star)
#         for point in range(len(mass_temp)):
#             rho_temp[point] = density(mass_temp[point], rad_temp[point])

#         rho_dist[star] = rho_temp
#         mass_dist[star] = mass_temp
#         rad_dist[star] = rad_temp


#     return rho_dist, mass_dist, rad_dist

In [261]:
# def density(mass, radius):
#     """Get density of sphere given mass and radius.

#     Parameters
#     ----------
#     mass: float
#         Mass of sphere (kg)
#     radius: float
#         Radius of sphere (m)

#     Returns
#     rho: float
#         Density of sphere (kg*m^-3)
#     """

#     rho = mass/((4.0/3.0)*np.pi*radius**3)
#     return rho

In [262]:
import photoeccentric as ph

In [263]:
rho, mass, radius = ph.find_density_dist_symmetric(ntargs, [mstar], [mstar_err], [rstar], [rstar_err])

100%|██████████| 1/1 [00:00<00:00, 34.15it/s]


In [264]:
np.mean(rho)

38949.79810546425

In [265]:
np.mean(mass)

3.0792045760938855e+29

In [266]:
np.mean(radius)

124663879.21450713

In [267]:
np.savetxt("jan29_21_rhos_" + '8733898' + ".csv", rho, delimiter=',')
#masspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")
#radiuspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")

In [268]:
mstar

0.15543012670105222

In [269]:
rstar

0.17888163900857723